In [1]:
################################################
### all the packages we need are defined here ##
################################################
using Logging
using Dates
using CSV
using DataFrames
using JLD2
using FileIO
using DataFrames
using Plots 
using CSV
using Distributions
using Random
using Tables
using DelimitedFiles
using BSON: @save, @load

In [2]:
mutable struct cell
    generation::Int64
    ID::Int64
    motherCellID::Int64
    founderCellID::Int64
    ODEFile::String
    fate::Int64 #0 = unknown, 1 = division, 2 = death
    tBirth::Float64
    tDeath::Float64
    solutionObject #the entire solution object. If you don't need this comment it out for efficiency.
    y0::Matrix{Float64} #initial conditions.
    yCoord::Float64 #for plotting lineage trees
    leftDaughterID::Int64
    rightDaughterID::Int64
    parameterList
    parameterNameList::Vector{String}
    cellsRepresented::Int64
    #newSwitch::Int64
end    

In [3]:
## add path to folder containing a cell from a recent simulation that will be used to get column names
folder = #pathToFolder

function createEmptyCellDF(folder)
    cell_df=DataFrame()

    @load folder*"/cell_1.jld2" thisCell
    sixhr_df = thisCell.solutionObject[(thisCell.solutionObject.timestamp .== 361.0), :]
    sixhr_df = sixhr_df[!, 2:195]
    # add patient ID
    insertcols!(sixhr_df, 1, :patient_ID => "DLBCL-LS1065")
    delete!(sixhr_df, [1])
    return(sixhr_df)
end

createEmptyCellDF (generic function with 1 method)

In [4]:
function getFounderCells(files, folder, sixhour_df, condition)
    for file in files
        count=length(files)
        wrong=0
        correctCell="cell_1.jld2"
        try
            @load folder*"/"*file*"/"*condition*"/processedCells/cell_1.jld2" thisCell 
            if thisCell.founderCellID!=1 || thisCell.generation!=1
                println("cell missmatch in patient: "*file)
                println(string(thisCell.founderCellID))
                println(string(thisCell.generation))
                thisPatFiles=readdir(folder*"/"*file*"/"*condition*"/processedCells/")
                for thisPatFile in thisPatFiles
                    @load folder*"/"*file*"/"*condition*"/processedCells/"*thisPatFile thisCell 
                    if thisCell.founderCellID==1 && thisCell.generation==1
                        println("found true cell 1: "*thisPatFile)
                        correctCell=thisPatFile
                        continue
                    end
                end
                wrong=wrong+1
            end
       @load folder*"/"*file*"/"*condition*"/processedCells/"*correctCell thisCell 
        temp_df = thisCell.solutionObject[(thisCell.solutionObject.timestamp .== 361.0), :]
        temp_df = temp_df[!, 2:195]
        # add patient ID
        insertcols!(temp_df, 1, :patient_ID => file)
        append!(sixhour_df, temp_df)
        catch
            println("couldn't process patient: "*file)
        end
    end
    return(sixhour_df)
end

getFounderCells (generic function with 1 method)

In [9]:
data_6h = createEmptyCellDF(folder)

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [10]:
# read the directory where the simulation data is saved to generate a list of patient IDs
files = readdir(folder)

95-element Vector{String}:
 "DIRN3501"
 "DIRN3502"
 "DIRN3503"
 "DIRN3504"
 "DIRN3505"
 "DIRN3506"
 "DIRN3507"
 "DIRN3510"
 "DIRN3512"
 "DIRN3513"
 "DIRN3514"
 "DIRN3517"
 "DIRN3521"
 ⋮
 "DIRN3668"
 "DIRN3672"
 "DIRN3676"
 "DIRN3678"
 "DIRN3680"
 "DIRN3681"
 "DIRN3684"
 "DIRN3686"
 "DIRN3691"
 "DIRN3694"
 "DIRN3696"
 "DIRN3697"

In [12]:
data_6h = getFounderCells(files, folder, data_comb_6h, "all_mut")

,patient_ID,cMyct,RelAnp50n,cRelnp50n,Bcl2t,Cycdt,tcRel,RelAnp52n
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,DIRN3501,6.80517,15.7504,5.55126,88.5857,0.100763,0.0386653,1.46893
2,DIRN3502,10.4532,24.1287,10.4459,88.2643,0.144227,0.0440746,1.86023
3,DIRN3503,8.41872,19.6414,7.6262,74.8072,0.120168,0.0413367,1.63954
4,DIRN3504,8.41836,19.6405,7.62512,104.09,0.120166,0.0413075,1.63951
5,DIRN3505,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959
6,DIRN3506,6.80517,15.7504,5.55126,88.5857,0.100763,0.0386653,1.46893
7,DIRN3507,15.1088,33.2207,18.0432,121.739,0.20068,0.0454401,2.46057
8,DIRN3510,6.80562,15.7503,5.55301,64.4727,0.100766,0.0386937,1.46884
9,DIRN3512,8.41863,19.6416,7.62555,74.8067,0.120167,0.0413075,1.63959


In [13]:
# write data to disk
CSV.write("data_6h.csv", data_6h)

"DanCT_combined_6h.csv"

In [60]:
files = unique(C2_single_strict.ParamAmount)

16-element Vector{String}:
 "k1_s_Blimp1-differentiation_0.5"
 "k1_mycTsl-linking_1.5"
 "nik_deg_mod-NFkB_0.5"
 "k2_scalingNumber-NFkB_1.5"
 "k1_bcl2Exp-linking_1.5"
 "k1_26-cellCycle_0.5"
 "kmax_IkBaSynth-NFkB_0.5"
 "k1_s_IRF4-linking_0.5"
 "k1_76-apoptosis_0.5"
 "k1_e_pax5-differentiation_0.5"
 "k1_s_Bcl6-differentiation_1.5"
 "k2_5-cellCycle_0.5"
 "k1_s_IRF4-linking_1.5"
 "k1_mycExp-linking_1.5"
 "k1_19-cellCycle_0.5"
 "k1_bcl2Exp-linking_0.5"

In [71]:
### assign 6h data to patients with single mutation
    npats_cumulative = 0
for i in 1:nrow(C2_single_6hrdf)
    param_df = DataFrame(C2_single_6hrdf[i, :])
    patients_df = C2_single_strict[(C2_single_strict.ParamAmount .== param_df.patient_ID),:]
    npats = nrow(patients_df)
    repeat!(param_df, npats)
    param_df[:, 1] = patients_df.PID
    append!(C2_strict_single_6h, param_df)
    npats_cumulative = npats_cumulative + npats
    println(npats_cumulative)
end

15
16
52
146
200
225
243
258
259
261
262
310
311
314
315
